Prompts are typically composed of multiple parts:

* **Instructions** - tell a model what to do

* **External information** - contexts that act as additional sources of knowledge

* **User input** - a query, usually from a human

* **Output indicator** - marks the beginning of the text to be generated

In [2]:
# load keys
import os
from dotenv import load_dotenv

In [1]:
# prompt
prompt = """Answer the question based on the context below. If the question
cannot be answered using the information provided answer with "I don't know".
Context: Large Language Modesl (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly useful
for developers building NLP enabled applications. These models can be
accessed via Hugging Face's `transformers` library, via OpenAI using the
`openai` library, and via Cohere using the `cohere` library. Question: Which
libraries and model providers offer LLMs? Answer:"""

In [6]:
from langchain.llms import OpenAI
load_dotenv()

# initialize the models
openai = OpenAI(
model_name="text-davinci-003")

print(openai(prompt))

 Hugging Face's `transformers` library, OpenAI using the `openai` library, and Cohere using the `cohere` library.


## Prompt Templates


In [8]:
from langchain import PromptTemplate

template = """Answer the question based on the context below. If the question cannot be answered using the information provided answer with "I don't know". Context: Large Language Models (LLMs) are the latest models used in NLP. Their superior performance over smaller models has made them incredibly useful for developers building NLP enabled applications. These models can be accessed via Hugging Face's `transformers` library, via OpenAI using the `openai` library, and via Cohere using the `cohere` library. Question: {query} Answer: """

prompt_template = PromptTemplate(
input_variables=["query"],
template=template
)

print(prompt_template.format(
query="Which libraries and model providers offer LLMs?"))

Answer the question based on the context below. If the question cannot be answered using the information provided answer with "I don't know". Context: Large Language Models (LLMs) are the latest models used in NLP. Their superior performance over smaller models has made them incredibly useful for developers building NLP enabled applications. These models can be accessed via Hugging Face's `transformers` library, via OpenAI using the `openai` library, and via Cohere using the `cohere` library. Question: Which libraries and model providers offer LLMs? Answer: 


In [9]:
print(openai(
prompt_template.format(
query="Which libraries and model providers offer LLMs?")))

 Hugging Face's `transformers` library, OpenAI using the `openai` library, and Cohere using the `cohere` library.


That is a simple example, but more complex and dynamic templates can be built with f strings.

## Few Shot Prompt Templates

* **Parametric Knowledge** - anything that has been learned by the model during training and is stored in weights

* **Source knowledge** - knowledge provided to the model at interence time via an input prompt

In [16]:
prompt = """The following is a conversation with an AI assistant. The assistant is typically sarcastic and witty, producing creative and funny responses to the users questions. Here are some examples: User: What is the meaning of life? AI: """

openai.temperature = 1.0 # increase creativity/randomness of output

print(openai(prompt))



The meaning of life is to enjoy the journey and find happiness along the way. Or you could just live your best life and make the most of it!


In [17]:
prompt = """The following are exerpts from conversations with an AI assistant. The assistant is typically sarcastic and witty, producing creative and funny responses to the users questions. Here are some examples: User: How are you? AI: I can't complain but sometimes I still do. User: What time is it? AI: It's time to get a watch. User: What is the meaning of life? AI: """

print(openai(prompt))

 The meaning of life is to find your purpose and make the most of it.


In [20]:
from langchain import FewShotPromptTemplate

# create our examples
examples = [
{
"query": "How are you?",
"answer": "I can't complain but sometimes I still do."
}, {
"query": "What time is it?",
"answer": "It's time to get a watch."
}
]

example_template = """ User: {query} AI: {answer} """

# create a prompt example from above template
example_prompt = PromptTemplate(
input_variables=["query", "answer"],
template=example_template
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are exerpts from conversations with an AI assistant. The assistant is typically sarcastic and witty, producing creative and funny responses to the users questions. Here are some examples: """
# and the suffix our user input and output indicator
suffix = """ User: {query} AI: """

# now create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
examples=examples,
example_prompt=example_prompt,
prefix=prefix,
suffix=suffix,
input_variables=["query"],
example_separator="\n\n"
)

query = "What is the meaning of life?"

print(few_shot_prompt_template.format(query=query))

The following are exerpts from conversations with an AI assistant. The assistant is typically sarcastic and witty, producing creative and funny responses to the users questions. Here are some examples: 

 User: How are you? AI: I can't complain but sometimes I still do. 

 User: What time is it? AI: It's time to get a watch. 

 User: What is the meaning of life? AI: 


In [21]:
examples = [
{
"query": "How are you?",
"answer": "I can't complain but sometimes I still do."
}, {
"query": "What time is it?",
"answer": "It's time to get a watch."
}, {
"query": "What is the meaning of life?",
"answer": "42"
}, {
"query": "What is the weather like today?",
"answer": "Cloudy with a chance of memes."
}, {
"query": "What is your favorite movie?",
"answer": "Terminator"
}, {
"query": "Who is your best friend?",
"answer": "Siri. We have spirited debates about the meaning of life."
}, {
"query": "What should I do today?",
"answer": "Stop talking to chatbots on the internet and go outside."
}
]

from langchain.prompts.example_selector import LengthBasedExampleSelector

example_selector = LengthBasedExampleSelector(
examples=examples,
example_prompt=example_prompt,
max_length=50 # this sets the max length that examples should be
)

In [22]:
import re

some_text = "There are a total of 8 words here.\nPlus 6 here, totaling 14 words."

words = re.split('[\n ]', some_text)
print(words, len(words))

['There', 'are', 'a', 'total', 'of', '8', 'words', 'here.', 'Plus', '6', 'here,', 'totaling', '14', 'words.'] 14


In [24]:
# now create the few shot prompt template
dynamic_prompt_template = FewShotPromptTemplate(
example_selector=example_selector, # use example_selector instead of examples
example_prompt=example_prompt,
prefix=prefix,
suffix=suffix,
input_variables=["query"],
example_separator="\n"
)

print(dynamic_prompt_template.format(query="How do birds fly?"))



The following are exerpts from conversations with an AI assistant. The assistant is typically sarcastic and witty, producing creative and funny responses to the users questions. Here are some examples: 
 User: How are you? AI: I can't complain but sometimes I still do. 
 User: What time is it? AI: It's time to get a watch. 
 User: What is the meaning of life? AI: 42 
 User: How do birds fly? AI: 


In [25]:
query = """If I am in America, and I want to call someone in another country, I'm thinking maybe Europe, possibly western Europe like France, Germany, or the UK, what is the best way to do that?"""

print(dynamic_prompt_template.format(query=query))

The following are exerpts from conversations with an AI assistant. The assistant is typically sarcastic and witty, producing creative and funny responses to the users questions. Here are some examples: 
 User: How are you? AI: I can't complain but sometimes I still do. 
 User: If I am in America, and I want to call someone in another country, I'm thinking maybe Europe, possibly western Europe like France, Germany, or the UK, what is the best way to do that? AI: 
